# Loading, computting and formatting the data 

In [ ]:
# Import necessary libraries
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import pickle
import os

import sys
sys.path.append('../utils')
from get_distance import compute_distance_matrix


In [ ]:
# Define file paths
pickle_filename = '../data/distance_matrix.pkl'
csv_filename = '../data/airports.csv'

# Check if the pickle file exists
if os.path.exists(pickle_filename):
    # Load the distance matrix from the pickle file
    with open(pickle_filename, 'rb') as f:
        distance_matrix = pickle.load(f)
    print("Loaded distance matrix from pickle file.")
else:
    # If the pickle file doesn't exist, compute the distance matrix
    print("Pickle file not found. Computing distance matrix...")
    distance_matrix = compute_distance_matrix(pickle_filename, csv_filename)

# Now you can use the distance_matrix in the rest of your code
print("Distance matrix is ready.")

In [ ]:
# Load the airports data
airports_file = '../data/airports.csv'
airports_df = pd.read_csv(airports_file)
print("Loaded airports data.")

In [ ]:
# Define the sets
N = airports_df[~airports_df['airport_code'].isin(['JFK', 'LAX'])]['airport_code'].tolist()  # Set of countries (excluding JFK and LAX)
V = N + ['JFK', 'LAX']  # Set of all nodes including JFK and LAX

In [ ]:
# Extract the distances c_{ij} for all pairs in V
c = {}
for i in V:
    c[i] = {}
    for j in V:
        if i != j:
            c[i][j] = distance_matrix.loc[i, j]
        else:
            c[i][j] = 0.0  # Distance to self is zero
            
print("Building optimization MTZ model...")

# Problem 1 with MTZ subtour elimination

In [ ]:
# Create the optimization model
model_1 = gp.Model("Problem1")

In [ ]:
# Variables
x = model_1.addVars(V, V, vtype=GRB.BINARY, name="x")
u = model_1.addVars(V, vtype=GRB.INTEGER, name="u")

In [ ]:
# Set the objective function
obj = gp.quicksum(c[i][j] * x[i, j] for i in V for j in V if i != j)
model_1.setObjective(obj, GRB.MINIMIZE)

In [ ]:
# Constraints
# 1. Departure from JFK
model_1.addConstr(gp.quicksum(x['JFK', j] for j in V if j != 'JFK') == 1, name="Depart_JFK")

In [ ]:
# 2. Arrival at LAX
model_1.addConstr(gp.quicksum(x[i, 'LAX'] for i in V if i != 'LAX') == 1, name="Arrive_LAX")

In [ ]:
# 3. Flow conservation for each country in N
for i in N:
    # Exactly one departure from country i
    model_1.addConstr(gp.quicksum(x[i, j] for j in V if j != i) == 1, name=f"Depart_{i}")
    # Exactly one arrival to country i
    model_1.addConstr(gp.quicksum(x[j, i] for j in V if j != i) == 1, name=f"Arrive_{i}")

In [ ]:
# 4. Subtour Elimination Constraints (MTZ)
num_nodes = len(N)
model_1.addConstr(u['JFK'] == 1, name="Position_JFK")
model_1.addConstr(u['LAX'] == num_nodes + 1, name="Position_LAX")
for i in N:
    model_1.addConstr(u[i] >= 1, name=f"MinPos_{i}")
    model_1.addConstr(u[i] <= num_nodes, name=f"MaxPos_{i}")
for i in V:
    for j in V:
        if i != j and i != 'LAX' and j != 'JFK':
            model_1.addConstr(
                u[i] - u[j] + num_nodes * x[i, j] <= num_nodes - 1,
                name=f"Subtour_{i}_{j}"
            )

In [ ]:
# Optimize the model
print("Optimizing the model...")
model_1.optimize()

In [ ]:
# Extract and print the solution
if model_1.status == GRB.OPTIMAL:
    print(f"\nOptimal objective value: {model_1.objVal:.2f} km")
    solution = model_1.getAttr('x', x)
    # Reconstruct the route
    route = ['JFK']
    next_node = 'JFK'
    while True:
        for j in V:
            if next_node != j and solution[next_node, j] > 0.5:
                route.append(j)
                next_node = j
                break
        if next_node == 'LAX':
            break
    print(f"Optimal Route: {' -> '.join(route)}")
else:
    print("No optimal solution found.")

# Problem 1 - revisit airports if it minimizes the total distance

In [ ]:
# Create the optimization model
modelNoMTZ = gp.Model("Problem1_LazyConstraints")

# Variables
x = modelNoMTZ.addVars(V, V, vtype=GRB.BINARY, name="x")

# Set the objective function
obj = gp.quicksum(c[i][j] * x[i, j] for i in V for j in V if i != j)
modelNoMTZ.setObjective(obj, GRB.MINIMIZE)

In [ ]:
# Constraints
# 1. Departure from JFK
modelNoMTZ.addConstr(gp.quicksum(x['JFK', j] for j in V if j != 'JFK') == 1, name="Depart_JFK")

# 2. Arrival at LAX
modelNoMTZ.addConstr(gp.quicksum(x[i, 'LAX'] for i in V if i != 'LAX') == 1, name="Arrive_LAX")

# 3. Flow conservation for each country in N
for i in N:
    # Exactly one departure from country i
    modelNoMTZ.addConstr(gp.quicksum(x[i, j] for j in V if j != i) == 1, name=f"Depart_{i}")
    # Exactly one arrival to country i
    modelNoMTZ.addConstr(gp.quicksum(x[j, i] for j in V if j != i) == 1, name=f"Arrive_{i}")


# for i in N:
#     inflow = gp.quicksum(x[j, i] for j in V if j != i)
#     outflow = gp.quicksum(x[i, j] for j in V if j != i)
#     modelNoMTZ.addConstr(inflow >= 1, name=f"AtLeastOneArrival_{i}")
#     modelNoMTZ.addConstr(outflow >= 1, name=f"AtLeastOneDeparture_{i}")
#     modelNoMTZ.addConstr(inflow == outflow, name=f"FlowConservation_{i}")

# # Special cases for JFK and LAX
# # JFK: Net outflow is 1 (start node)
# modelNoMTZ.addConstr(
#     gp.quicksum(x['JFK', j] for j in V if j != 'JFK') - gp.quicksum(x[j, 'JFK'] for j in V if j != 'JFK') == 1,
#     name="FlowConservation_JFK"
# )

# # LAX: Net inflow is 1 (end node)
# modelNoMTZ.addConstr(
#     gp.quicksum(x[j, 'LAX'] for j in V if j != 'LAX') - gp.quicksum(x['LAX', j] for j in V if j != 'LAX') == 1,
#     name="FlowConservation_LAX"
# )

modelNoMTZ.update()

In [ ]:
# Function to find subtours
def get_subtours(selected):
    # Build a dictionary to represent the graph
    from collections import defaultdict
    graph = defaultdict(list)
    for i, j in selected:
        graph[i].append(j)
    # Find subtours
    unvisited = set(V)
    subtours = []
    while unvisited:
        current = unvisited.pop()
        tour = [current]
        while True:
            neighbors = graph[current]
            for neighbor in neighbors:
                if neighbor in unvisited:
                    unvisited.remove(neighbor)
                    tour.append(neighbor)
                    current = neighbor
                    break
            else:
                break
        subtours.append(tour)
    return subtours

In [ ]:
# Set the parameters to use lazy constraints
modelNoMTZ.Params.LazyConstraints = 1

# Callback function to eliminate subtours
def subtour_elimination(modelNoMTZ, where):
    if where == GRB.Callback.MIPSOL:
        # Get the solution
        vals = modelNoMTZ.cbGetSolution(modelNoMTZ._x)
        # Build a list of edges selected in the solution
        selected = gp.tuplelist((i, j) for i in V for j in V if i != j and vals[i, j] > 0.5)
        # Find the subtours
        tours = get_subtours(selected)
        # If more than one subtour exists, add subtour elimination constraints
        if len(tours) > 1:
            for tour in tours:
                if 'JFK' in tour and 'LAX' in tour and len(tour) == len(V):
                    continue  # Main tour found
                # Add subtour elimination constraint
                modelNoMTZ.cbLazy(gp.quicksum(x[i, j] for i in tour for j in tour if i != j) <= len(tour) - 1)

In [ ]:
# Optimize the model with the callback
print("Optimizing the model No MTZ...")
modelNoMTZ._x = x
modelNoMTZ.optimize(subtour_elimination)

In [ ]:
# Extract and print the solution
if modelNoMTZ.status == GRB.OPTIMAL:
    print(f"\nOptimal objective value: {modelNoMTZ.objVal:.2f} km")
    solution = modelNoMTZ.getAttr('x', x)
    # Reconstruct the route
    selected = gp.tuplelist((i, j) for i in V for j in V if i != j and solution[i, j] > 0.5)
    route = ['JFK']
    next_node = 'JFK'
    while next_node != 'LAX':
        for i, j in selected:
            if i == next_node:
                route.append(j)
                next_node = j
                break
    print(f"Optimal Route: {' -> '.join(route)}")
else:
    print("No optimal solution found.")

In [ ]:
# Now, plot the route on a map
print("\nPlotting the route on a map...")

# Get latitude and longitude for each airport in the route
coordinates = []
for code in route:
    airport = airports_df[airports_df['airport_code'] == code].iloc[0]
    lat = airport['latitude_deg']
    lon = airport['longitude_deg']
    coordinates.append((lon, lat))  # Note: longitude first for plotting

# Create the map
fig = plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
ax.coastlines()

# Plot the route
lons, lats = zip(*coordinates)
ax.plot(lons, lats, color='blue', linewidth=2, marker='o', transform=ccrs.Geodetic())

# Annotate airport codes
for lon, lat, code in zip(lons, lats, route):
    ax.text(lon + 1, lat + 1, code, transform=ccrs.Geodetic())

# Set title
ax.set_title('Optimal Route for Health Commodity Shipment')

plt.savefig('../output/problem1.png', dpi=300)
# Show the plot
plt.show()

# Problem 2 - ALWAYS 2 planes

In [ ]:
# Import necessary libraries
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [ ]:
# Create the optimization model
model_2 = gp.Model("Problem2_TwoPlanes")

# Variables
x = model_2.addVars(V, V, K, vtype=GRB.BINARY, name="x")
y = model_2.addVars(N, vtype=GRB.BINARY, name="y")

# Set the objective function
obj = gp.quicksum(c[i][j] * x[i, j, k] for k in K for i in V for j in V if i != j)
model_2.setObjective(obj, GRB.MINIMIZE)

In [ ]:

# Constraints

# 1. Each country is visited exactly once
for i in N:
    model_2.addConstr(
        gp.quicksum(x[i, j, k] for k in K for j in V if j != i) == 1,
        name=f"VisitOnce_{i}"
    )

# 2. Plane assignment constraints
for i in N:
    # Incoming arcs
    model_2.addConstr(
        gp.quicksum(x[j, i, 'JFK'] for j in V if j != i) == y[i],
        name=f"Assign_JFK_in_{i}"
    )
    model_2.addConstr(
        gp.quicksum(x[j, i, 'LAX'] for j in V if j != i) == 1 - y[i],
        name=f"Assign_LAX_in_{i}"
    )
    # Outgoing arcs
    model_2.addConstr(
        gp.quicksum(x[i, j, 'JFK'] for j in V if j != i) == y[i],
        name=f"Assign_JFK_out_{i}"
    )
    model_2.addConstr(
        gp.quicksum(x[i, j, 'LAX'] for j in V if j != i) == 1 - y[i],
        name=f"Assign_LAX_out_{i}"
    )

# 3. Departure and return constraints for each plane
for k in K:
    # Departure from warehouse
    model_2.addConstr(
        gp.quicksum(x[k, j, k] for j in V if j != k) == 1,
        name=f"Depart_{k}"
    )
    # Return to warehouse
    model_2.addConstr(
        gp.quicksum(x[i, k, k] for i in V if i != k) == 1,
        name=f"Return_{k}"
    )

# 4. Flow conservation for each plane
for k in K:
    for i in V:
        if i != k:
            inflow = gp.quicksum(x[j, i, k] for j in V if j != i)
            outflow = gp.quicksum(x[i, j, k] for j in V if j != i)
            model_2.addConstr(inflow == outflow, name=f"FlowConservation_{i}_{k}")
        else:
            # For the warehouse nodes
            inflow = gp.quicksum(x[j, i, k] for j in V if j != i)
            outflow = gp.quicksum(x[i, j, k] for j in V if j != i)
            model_2.addConstr(outflow - inflow == 0, name=f"FlowConservation_{i}_{k}")

# 5. Prevent planes from visiting the other warehouse
# Plane starting at JFK should not visit LAX
for i in V:
    model_2.addConstr(x[i, 'LAX', 'JFK'] == 0, name=f"NoVisit_LAX_by_JFK_{i}")
    model_2.addConstr(x['LAX', i, 'JFK'] == 0, name=f"NoVisitFrom_LAX_by_JFK_{i}")
# Plane starting at LAX should not visit JFK
for i in V:
    model_2.addConstr(x[i, 'JFK', 'LAX'] == 0, name=f"NoVisit_JFK_by_LAX_{i}")
    model_2.addConstr(x['JFK', i, 'LAX'] == 0, name=f"NoVisitFrom_JFK_by_LAX_{i}")

# 6. Prevent self-loops
for k in K:
    for i in V:
        model_2.addConstr(x[i, i, k] == 0, name=f"NoSelfLoop_{i}_{k}")

# 7. Optional: Balance the number of countries assigned to each plane
num_countries = len(N)
min_countries_per_plane = num_countries // 2  # Adjust as necessary
model_2.addConstr(
    gp.quicksum(y[i] for i in N) >= min_countries_per_plane,
    name="MinCountries_JFK"
)
model_2.addConstr(
    gp.quicksum(1 - y[i] for i in N) >= min_countries_per_plane,
    name="MinCountries_LAX"
)

model_2.update()

In [ ]:
# Subtour elimination via Lazy Constraints
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get the solution
        vals = model.cbGetSolution(model._x)
        for k in K:
            # Build a list of edges selected in the solution for plane k
            selected = gp.tuplelist(
                (i, j) for i in V for j in V if i != j and vals[i, j, k] > 0.5
            )
            # Find the subtours
            tours = get_subtours(selected)
            # If more than one subtour exists, add subtour elimination constraints
            if len(tours) > 1:
                for tour in tours:
                    if k in tour and len(tour) == len(selected) + 1:
                        continue  # Main tour found
                    # Add subtour elimination constraint
                    model.cbLazy(
                        gp.quicksum(x[i, j, k] for i in tour for j in tour if i != j) <= len(tour) - 1
                    )

def get_subtours(selected):
    # Build a dictionary to represent the graph
    from collections import defaultdict
    graph = defaultdict(list)
    for i, j in selected:
        graph[i].append(j)
    # Find subtours
    unvisited = set(graph.keys())
    subtours = []
    while unvisited:
        current = unvisited.pop()
        tour = [current]
        while True:
            neighbors = graph[current]
            found = False
            for neighbor in neighbors:
                if neighbor in unvisited:
                    unvisited.remove(neighbor)
                    tour.append(neighbor)
                    current = neighbor
                    found = True
                    break
            if not found:
                break
        subtours.append(tour)
    return subtours

# Set the parameters to use lazy constraints
model_2.Params.LazyConstraints = 1

# Set a time limit if the model takes too long to solve... :(
# model.Params.TimeLimit = 3600  # Time limit in seconds

In [ ]:
# Optimize the model with the callback
print("Optimizing the model...")
model_2._x = x
model_2.optimize(subtour_elimination)

In [ ]:
# Extract and print the solution
if model_2.status == GRB.OPTIMAL or model_2.status == GRB.TIME_LIMIT:
    print(f"\nOptimal objective value: {model_2.objVal:.2f} km")
    solution_x = model_2.getAttr('x', x)
    solution_y = model_2.getAttr('x', y)

    # Build routes for each plane
    routes = {}
    for k in K:
        selected = gp.tuplelist(
            (i, j) for i in V for j in V if i != j and solution_x[i, j, k] > 0.5
        )
        route = [k]
        next_node = k
        while True:
            for i, j in selected:
                if i == next_node:
                    route.append(j)
                    next_node = j
                    break
            else:
                break
            if next_node == k:
                break
        routes[k] = route
        print(f"\nOptimal Route for plane starting at {k}: {' -> '.join(route)}")
else:
    print("No optimal solution found.")

In [ ]:
# Now, plot the routes on a map
print("\nPlotting the routes on a map...")

# Get coordinates for each airport
coordinates = {}
for code in V:
    airport = airports_df[airports_df['airport_code'] == code].iloc[0]
    lat = airport['latitude_deg']
    lon = airport['longitude_deg']
    coordinates[code] = (lon, lat)  # longitude, latitude# Create the map
fig = plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
ax.coastlines()

# Plot the routes for each plane
colors = {'JFK': 'blue', 'LAX': 'red'}
for k in K:
    if k in routes:
        route = routes[k]
        lons = [coordinates[code][0] for code in route]
        lats = [coordinates[code][1] for code in route]
        ax.plot(
            lons, lats, color=colors[k], linewidth=2, marker='o',
            transform=ccrs.Geodetic(), label=f"Plane from {k}"
        )
        # Annotate airport codes
        for lon, lat, code in zip(lons, lats, route):
            ax.text(lon + 1, lat + 1, code, transform=ccrs.Geodetic())
    else:
        print(f"No route found for plane starting at {k}")

# Set title and legend
ax.set_title('Optimal Routes for Two Planes')
ax.legend()

# Save the plot if desired
plt.savefig('../output/problem2.png', dpi=300)

# Show the plot
plt.show()

# Problem 2 - We can have only one plane if optimal

In [ ]:
# Create the optimization model
model_2_optional = gp.Model("Problem2_OptionalPlanes")

# Variables
x = model_2_optional.addVars(V, V, K, vtype=GRB.BINARY, name="x")
y = model_2_optional.addVars(N, vtype=GRB.BINARY, name="y")
u = model_2_optional.addVars(K, vtype=GRB.BINARY, name="u")  # Plane usage variables

# Set the objective function (no fixed cost per plane)
obj = gp.quicksum(c[i][j] * x[i, j, k] for k in K for i in V for j in V if i != j)
model_2_optional.setObjective(obj, GRB.MINIMIZE)

In [ ]:
# Constraints

# 1. Each country is visited exactly once
for i in N:
    model_2_optional.addConstr(
        gp.quicksum(x[i, j, k] for k in K for j in V if j != i) == 1,
        name=f"VisitOnce_{i}"
    )

# 2. Plane assignment constraints
for i in N:
    # Ensure that countries are assigned to planes only if the planes are used
    model_2_optional.addConstr(y[i] <= u['JFK'], name=f"Assign_JFK_u_{i}")
    model_2_optional.addConstr((1 - y[i]) <= u['LAX'], name=f"Assign_LAX_u_{i}")

    # Incoming arcs
    model_2_optional.addConstr(
        gp.quicksum(x[j, i, 'JFK'] for j in V if j != i) == y[i],
        name=f"Assign_JFK_in_{i}"
    )
    model_2_optional.addConstr(
        gp.quicksum(x[j, i, 'LAX'] for j in V if j != i) == 1 - y[i],
        name=f"Assign_LAX_in_{i}"
    )
    # Outgoing arcs
    model_2_optional.addConstr(
        gp.quicksum(x[i, j, 'JFK'] for j in V if j != i) == y[i],
        name=f"Assign_JFK_out_{i}"
    )
    model_2_optional.addConstr(
        gp.quicksum(x[i, j, 'LAX'] for j in V if j != i) == 1 - y[i],
        name=f"Assign_LAX_out_{i}"
    )

# 3. Departure and return constraints for each plane
for k in K:
    # Departure from warehouse
    model_2_optional.addConstr(
        gp.quicksum(x[k, j, k] for j in V if j != k) == u[k],
        name=f"Depart_{k}"
    )
    # Return to warehouse
    model_2_optional.addConstr(
        gp.quicksum(x[i, k, k] for i in V if i != k) == u[k],
        name=f"Return_{k}"
    )

# 4. Ensure that if a plane is not used, its arcs are not selected
for k in K:
    for i in V:
        for j in V:
            if i != j:
                model_2_optional.addConstr(x[i, j, k] <= u[k], name=f"UsePlane_{i}_{j}_{k}")

# 5. Flow conservation for each plane
for k in K:
    for i in V:
        inflow = gp.quicksum(x[j, i, k] for j in V if j != i)
        outflow = gp.quicksum(x[i, j, k] for j in V if j != i)
        model_2_optional.addConstr(inflow == outflow, name=f"FlowConservation_{i}_{k}")

# 6. Prevent planes from visiting the other warehouse
# Plane starting at JFK should not visit LAX
for i in V:
    model_2_optional.addConstr(x[i, 'LAX', 'JFK'] == 0, name=f"NoVisit_LAX_by_JFK_{i}")
    model_2_optional.addConstr(x['LAX', i, 'JFK'] == 0, name=f"NoVisitFrom_LAX_by_JFK_{i}")
# Plane starting at LAX should not visit JFK
for i in V:
    model_2_optional.addConstr(x[i, 'JFK', 'LAX'] == 0, name=f"NoVisit_JFK_by_LAX_{i}")
    model_2_optional.addConstr(x['JFK', i, 'LAX'] == 0, name=f"NoVisitFrom_JFK_by_LAX_{i}")

# 7. Prevent self-loops
for k in K:
    for i in V:
        model_2_optional.addConstr(x[i, i, k] == 0, name=f"NoSelfLoop_{i}_{k}")

In [ ]:
# Subtour elimination via Lazy Constraints
def subtour_elimination(model, where):
    if where == GRB.Callback.MIPSOL:
        # Get the solution for x variables
        vals = model.cbGetSolution(model._x)
        for k in K:
            # Get the value of u[k] directly
            u_k_value = model.cbGetSolution(u[k])
            if u_k_value < 0.5:
                continue  # Plane not used
            # Build a list of edges selected in the solution for plane k
            selected = gp.tuplelist(
                (i, j) for i in V for j in V if i != j and vals[i, j, k] > 0.5
            )
            if selected:
                # Find the subtours
                tours = get_subtours(selected)
                # If more than one subtour exists, add subtour elimination constraints
                if len(tours) > 1:
                    for tour in tours:
                        if k in tour and len(tour) == len(V):
                            continue  # Main tour found
                        # Add subtour elimination constraint
                        model.cbLazy(
                            gp.quicksum(x[i, j, k] for i in tour for j in tour if i != j) <= len(tour) - 1
                        )

def get_subtours(selected):
    # Build a dictionary to represent the graph
    from collections import defaultdict
    graph = defaultdict(list)
    for i, j in selected:
        graph[i].append(j)
    # Find subtours
    unvisited = set(graph.keys())
    subtours = []
    while unvisited:
        current = unvisited.pop()
        tour = [current]
        while True:
            neighbors = graph[current]
            found = False
            for neighbor in neighbors:
                if neighbor in unvisited:
                    unvisited.remove(neighbor)
                    tour.append(neighbor)
                    current = neighbor
                    found = True
                    break
            if not found:
                break
        subtours.append(tour)
    return subtours

# Set the parameters to use lazy constraints
model_2_optional.Params.LazyConstraints = 1

In [ ]:
# Optimize the model with the callback
print("Optimizing the model...")
model_2_optional._x = x
model_2_optional.optimize(subtour_elimination)

In [ ]:
# Extract and print the solution
if model_2_optional.status == GRB.OPTIMAL or model_2_optional.status == GRB.TIME_LIMIT:
    print(f"\nOptimal objective value: {model_2_optional.objVal:.2f} km")
    solution_x = model_2_optional.getAttr('x', x)
    solution_u = model_2_optional.getAttr('x', u)
    solution_y = model_2_optional.getAttr('x', y)

    # Build routes for each plane
    routes = {}
    for k in K:
        if solution_u[k] > 0.5:
            # Plane k is used
            selected = gp.tuplelist(
                (i, j) for i in V for j in V if i != j and solution_x[i, j, k] > 0.5
            )
            route = [k]
            next_node = k
            while True:
                for i, j in selected:
                    if i == next_node:
                        route.append(j)
                        next_node = j
                        break
                else:
                    break
                if next_node == k:
                    break
            routes[k] = route
            print(f"\nOptimal Route for plane starting at {k}: {' -> '.join(route)}")
        else:
            print(f"\nPlane starting at {k} is not used.")
else:
    print("No optimal solution found.")

In [ ]:
# Now, plot the routes on a map
print("\nPlotting the routes on a map...")

# Get coordinates for each airport
coordinates = {}
for code in V:
    airport = airports_df[airports_df['airport_code'] == code].iloc[0]
    lat = airport['latitude_deg']
    lon = airport['longitude_deg']
    coordinates[code] = (lon, lat)  # longitude, latitude

# Create the map
fig = plt.figure(figsize=(15, 10))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.stock_img()
ax.coastlines()

# Plot the routes for each plane
colors = {'JFK': 'blue', 'LAX': 'red'}
for k in K:
    if k in routes:
        route = routes[k]
        lons = [coordinates[code][0] for code in route]
        lats = [coordinates[code][1] for code in route]
        ax.plot(
            lons, lats, color=colors[k], linewidth=2, marker='o',
            transform=ccrs.Geodetic(), label=f"Plane from {k}"
        )
        # Annotate airport codes
        for lon, lat, code in zip(lons, lats, route):
            ax.text(lon + 1, lat + 1, code, transform=ccrs.Geodetic())
    else:
        print(f"No route found for plane starting at {k}")

# Set title and legend
ax.set_title('Optimal Routes with Optional Plane Usage')
ax.legend()

# Save the plot if desired
plt.savefig('../output/problem2_optional_planes.png', dpi=300)

# Show the plot
plt.show()